## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-25-09-04-00 +0000,wsj,Periods of Risk Aversion Should Boost Treasury...,https://www.wsj.com/finance/jgb-futures-rise-a...
1,2026-02-25-09-04-00 +0000,wsj,Thailand’s Central Bank Surprises With Rate Cut,https://www.wsj.com/articles/thailands-central...
2,2026-02-25-09-00-00 +0000,cbc,Where is Canada's immigration minister? Commun...,https://www.cbc.ca/news/politics/lena-diab-imm...
3,2026-02-25-08-59-14 +0000,nypost,Virginia Dem Gov. Abigail Spanberger delivers ...,https://nypost.com/2026/02/25/us-news/virginia...
4,2026-02-25-08-55-55 +0000,bbc,Mandelson's lawyers say he was arrested over '...,https://www.bbc.com/news/articles/c05vey47yddo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2546/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
27,trump,78
28,union,37
175,new,31
86,year,21
29,address,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
66,2026-02-25-05-07-00 +0000,wsj,President Trump told a national audience in hi...,https://www.wsj.com/politics/policy/trump-hail...,228
119,2026-02-25-03-04-21 +0000,nyt,"“After four years, in which millions and milli...",https://www.nytimes.com/live/2026/02/25/us/fac...,194
131,2026-02-25-02-43-12 +0000,nypost,"From USA hockey team to 100-year-old veteran, ...",https://nypost.com/2026/02/24/us-news/usa-chan...,183
137,2026-02-25-02-28-12 +0000,nypost,Rep. Al Green booted from State of the Union 2...,https://nypost.com/2026/02/24/us-news/rep-al-g...,181
72,2026-02-25-04-57-00 +0000,wsj,President Trump delivered a State of the Union...,https://www.wsj.com/politics/policy/five-takea...,176


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
66,228,2026-02-25-05-07-00 +0000,wsj,President Trump told a national audience in hi...,https://www.wsj.com/politics/policy/trump-hail...
119,194,2026-02-25-03-04-21 +0000,nyt,"“After four years, in which millions and milli...",https://www.nytimes.com/live/2026/02/25/us/fac...
76,134,2026-02-25-04-39-24 +0000,nyt,“When I last spoke in the chamber 12 months ag...,https://www.nytimes.com/live/2026/02/25/us/fac...
86,92,2026-02-25-04-06-50 +0000,nyt,“They’ve killed and maimed thousands of Americ...,https://www.nytimes.com/live/2026/02/25/us/fac...
165,58,2026-02-24-23-54-43 +0000,nypost,Warner Bros. Discovery may upend Netflix deal ...,https://nypost.com/2026/02/24/media/warner-bro...
108,57,2026-02-25-03-24-12 +0000,nyt,“Under my just-enacted most favored nation agr...,https://www.nytimes.com/live/2026/02/25/us/fac...
100,55,2026-02-25-03-35-53 +0000,nyt,“I want to stop all payments to big insurance ...,https://www.nytimes.com/live/2026/02/24/us/tru...
101,55,2026-02-25-03-35-45 +0000,nyt,"“With the great Big Beautiful Bill, we gave yo...",https://www.nytimes.com/live/2026/02/25/us/fac...
359,51,2026-02-24-13-14-53 +0000,bbc,Ramaphosa thanks Putin for release of South Af...,https://www.bbc.com/news/articles/c2d1j7klzpgo...
239,49,2026-02-24-20-26-51 +0000,nypost,"Ozempic, Wegovy prices to drop up to 50% as No...",https://nypost.com/2026/02/24/business/ozempic...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
